In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
from PIL import Image, ImageDraw, ImageFont 
import random
import numpy as np
import os
from tqdm import tqdm
import copy
import re
from IPython.display import display, HTML
import rembg
import matplotlib.pyplot as plt
%matplotlib inline

colormap = ['blue','orange','green','purple','brown','pink','gray','olive','cyan','red',
            'lime','indigo','violet','aqua','magenta','coral','gold','tan','skyblue']

2024-07-01 14:19:09.458705: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 14:19:12.145046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 14:19:12.936150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 14:19:12.942869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 14:19:14.542482: I tensorflow/core/platform/cpu_feature_gua

## Load the model

In [3]:
model_id = 'microsoft/Florence-2-base-ft'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
# Code to extract
def extract_text(task_prompt, image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
        print(prompt)
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text, 
        task=task_prompt, 
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [6]:
# Extract necessary info from ocr response
def extract_info(labels):
    account_number_pattern_1 = re.compile(r'\b(?:\d{1,4}\s*){1,2}\d{2,4}\s?\d{2,4}\s?\d{2,4}\s?\d{1,4}\b')
    account_number_pattern_2 = r'\b\d+\b'
    validity_date_pattern = re.compile(r'\b(?:THRU)?\d{2}\s*/\s*\d{2}\b')
    name_pattern_1 = re.compile(r'^[A-Z.]{1,23}\s[A-Z. ]{1,23}$')
    
    result = {
        'account_number': None,
        'account_holder_name': None,
        'validity_date': None
    }
    splitted_acc_number = ""
    for label in labels:
        if account_number_pattern_1.match(label):
            label = label.replace(" ", "")
            if len(label) > 16:
                label = label[:16]
            formatted_label = ' '.join([label[i:i+4] for i in range(0, len(label), 4)])
            result['account_number'] = formatted_label
        if result['account_number'] is None or len(result['account_number']) < 19:
            numbers = re.findall(account_number_pattern_2, label)
            splitted_acc_number += ' '.join(numbers)
            
        if validity_date_pattern.search(label):
            match = validity_date_pattern.search(label).group(0)
            result['validity_date'] = re.sub(r'[^/\d]', '', match)
            
        if name_pattern_1.match(label) and not account_number_pattern_1.match(label) and not validity_date_pattern.match(label):
            result['account_holder_name'] = label
        
        # if result['account_holder_name'] is None and name_pattern_2.match(label) and not account_number_pattern_1.match(label) and not validity_date_pattern.match(label):
        #     label = re.sub(r'[^A-Z. ]', '', label)
        #     result['account_holder_name'] = label
            
    if result['account_number'] is None or len(result['account_number']) != 19:
        splitted_acc_number = splitted_acc_number.replace(" ", "")
        if len(splitted_acc_number) == 0:
            result['account_number'] = None
        if len(splitted_acc_number) > 16:
            splitted_acc_number = splitted_acc_number[:16]
        splitted_acc_number = ' '.join([splitted_acc_number[i:i+4] for i in range(0, len(splitted_acc_number), 4)])
        result['account_number'] = splitted_acc_number
    return result

In [4]:
# Display the image and results
def display_image_and_info(image_path, info, ocr_results):
    image = Image.open(image_path).convert('RGB')
    # display(image)
    html_content = f"""
    <div style="display: flex; flex-direction: column; align-items: flex-start;">
        <p><strong>Image Path:</strong> {image_path}</p>
        <div style="display: flex;">
            <div style="margin-right: 20px;">
                <img src="{image_path}" style="max-width: 300px; max-height: 300px;">
            </div>
            <div>
                <p><strong>OCR Labels:</strong> {', '.join(ocr_results['labels'])}</p>
                <p><strong>Account Number:</strong> {info['account_number']}</p>
                <p><strong>Account Holder Name:</strong> {info['account_holder_name']}</p>
                <p><strong>Validity Date:</strong> {info['validity_date']}</p>
            </div>
        </div>
    </div>
    """
    display(HTML(html_content))
    print("---"*30)

In [5]:
def get_input_images(folder_path):
    files = os.listdir(folder_path)
    image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
    image_paths = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image_paths.append(image_path)
    return image_paths

In [8]:
image_paths = get_input_images("images")
# image_paths = get_input_images("images-bg")
# image_paths = get_input_images("real_world_images")
# image_paths = get_input_images("scaled_img")
# image_paths = get_input_images("enhanced_img")
len(image_paths)

77

In [14]:
# OCR with region or OCR

task_prompt = '<OCR_WITH_REGION>'
# task_prompt = '<OCR>'

In [15]:
results = []
for image_path in tqdm(image_paths):
    image = Image.open(image_path).convert('RGB')
    results.append(extract_text(task_prompt, image))

100%|██████████| 77/77 [01:10<00:00,  1.09it/s]


In [16]:
cleaned_extraction = []
for i in range(len(results)):
    cleaned_text = extract_info(results[i]['<OCR_WITH_REGION>']['labels'])
    cleaned_extraction.append(cleaned_text)

In [17]:
for i in range(len(results)):
    display_image_and_info(image_paths[i], cleaned_extraction[i], results[i]['<OCR_WITH_REGION>'])

------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


...

------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------


In [2]:
import psutil
from signal import SIGKILL
 
for proc in psutil.process_iter(['pid', 'name']):
            try:
                if proc.info['pid'] == 1:
                    continue
                for conns in proc.connections(kind='inet'):
                    if conns.laddr.port in [8000]:
                        print(conns.laddr.port)
                        proc.send_signal(SIGKILL)
            except (psutil.NoSuchProcess, psutil.AccessDenied) as e:
                # logging.error(e)
                # print(e)
                pass